In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
import ddf_utils.ddf_reader as dr
from ddf_utils.str import format_float_sigfig, format_float_digits

In [3]:
dr.SEARCH_PATH = '../../../'

In [4]:
co2 = dr.ddf_datapoint('ddf--cdiac-co2', 
                       'total_carbon_emissions', 'nation,year')

In [5]:
pop = dr.ddf_datapoint('ddf--gapminder--population', 'population')

In [6]:
geo = dr.ddf_entities('ddf--gapminder--geo_entity_domain')['country']
nation = dr.ddf_entities('ddf--cdiac-co2')['nation']

In [7]:
co2.head()

,nation,year,total_carbon_emissions
0,afghanistan,1949,4.0
1,afghanistan,1950,23.0
2,afghanistan,1951,25.0
3,afghanistan,1952,25.0
4,afghanistan,1953,29.0


In [8]:
pop.head()

,country,year,population
0,abw,1800,19286.0
1,abw,1801,19286.0
2,abw,1802,19286.0
3,abw,1803,19286.0
4,abw,1804,19286.0


In [12]:
# see if we can make the nation align to gapminder geo domain

In [9]:
cdiac_nation_map = nation.to_dict('record')

In [10]:
cdiac_nation_map = dict([(x['nation'], x['name']) for x in cdiac_nation_map])

In [11]:
co2['name'] = co2.nation.map(lambda x: cdiac_nation_map[x])

In [12]:
co2.head()

,nation,year,total_carbon_emissions,name
0,afghanistan,1949,4.0,AFGHANISTAN
1,afghanistan,1950,23.0,AFGHANISTAN
2,afghanistan,1951,25.0,AFGHANISTAN
3,afghanistan,1952,25.0,AFGHANISTAN
4,afghanistan,1953,29.0,AFGHANISTAN


In [13]:
map_c = {}
search_cols = ['name', 'gapminder_list','alternative_1', 'alternative_2', 'alternative_3',
               'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name', 
               'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6'
              ]

for g in co2.name.unique():
    masks = []
    for c in search_cols:
        masks.append(geo[c].str.lower() == g.lower())
    
    map0 = masks[0]
    for m in masks[1:]:
        map0 = map0 | m
        
    filtered = geo[map0]
    if len(filtered) > 1:
        print('multiple entities found for '+g)
        print(filtered['country'].values)
    elif len(filtered) > 0:
        map_c[g] = filtered['country'].values[0]
    else:
        print('not found: ', g)

not found:  ANTARCTIC FISHERIES
not found:  BONAIRE, SAINT EUSTATIUS, AND SABA
not found:  DEMOCRATIC REPUBLIC OF VIETNAM
not found:  EAST & WEST PAKISTAN
not found:  FEDERATION OF MALAYA-SINGAPORE
not found:  FORMER PANAMA CANAL ZONE
not found:  FRENCH EQUATORIAL AFRICA
not found:  FRENCH INDO-CHINA
not found:  FRENCH WEST AFRICA
not found:  KUWAITI OIL FIRES
not found:  LEEWARD ISLANDS
not found:  NETHERLAND ANTILLES AND ARUBA
not found:  PACIFIC ISLANDS (PALAU)
not found:  PENINSULAR MALAYSIA
not found:  REPUBLIC OF SOUTH VIETNAM
not found:  RHODESIA-NYASALAND
not found:  RWANDA-URUNDI
not found:  RYUKYU ISLANDS
not found:  SABAH
not found:  SARAWAK
not found:  ST. KITTS-NEVIS-ANGUILLA
not found:  TANGANYIKA
not found:  ZANZIBAR


In [14]:
co2['geo'] = co2.name.map(lambda x: map_c[x] if x in map_c.keys() else np.nan)

# deal with the split/merge case

Here is how I do with split/merge:

merge: rename the nation name to target geo, then groupby geo/year and sum each group.

In [15]:
# vnm = DEMOCRATIC REPUBLIC OF VIETNAM + REPUBLIC OF SOUTH VIETNAM

select = (co2.name == 'DEMOCRATIC REPUBLIC OF VIETNAM') | (co2.name == 'REPUBLIC OF SOUTH VIETNAM')

# append new data to origin, with new geo.
to_append = co2.ix[select].copy()
to_append['geo'] = 'vnm'

co2 = co2.append(to_append, ignore_index=True)

In [16]:
# deu = FEDERAL REPUBLIC OF GERMANY + FORMER GERMAN DEMOCRATIC REPUBLIC

select = (co2.name == 'FEDERAL REPUBLIC OF GERMANY') | (co2.name == 'FORMER GERMAN DEMOCRATIC REPUBLIC')

to_append = co2.ix[select].copy()
to_append['geo'] = 'deu'

co2 = co2.append(to_append, ignore_index=True)

In [17]:
# yem = FORMER DEMOCRATIC YEMEN + FORMER YEMEN

select = (co2.name == 'FORMER DEMOCRATIC YEMEN') | (co2.name == 'FORMER YEMEN')

to_append = co2.ix[select].copy()
to_append['geo'] = 'yem'

co2 = co2.append(to_append, ignore_index=True)

In [18]:
# tza = TANGANYIKA + ZANZIBAR

select = (co2.name == 'TANGANYIKA') | (co2.name == 'ZANZIBAR')

to_append = co2.ix[select].copy()
to_append['geo'] = 'tza'

co2 = co2.append(to_append, ignore_index=True)

In [19]:
def split_with_ratio(data, ratio):
    new_data = []

    for idx, row in data.iterrows():
        year = row['year']

        for geo, r in ratio.items():
            new_data.append({'geo': geo, 
                           'total_carbon_emissions': row['total_carbon_emissions'] * r, 
                           'year': year
                          })

    res = pd.DataFrame.from_records(new_data).sort_values(by=['geo', 'year'])
    return res

In [20]:
# EAST & WEST PAKISTAN:  split with 1972 ratio of pak and bgd

pak = co2.ix[(co2.geo == 'pak') & (co2.year == 1972), 'total_carbon_emissions'].values[0]
bgd = co2.ix[(co2.geo == 'bgd') & (co2.year == 1972), 'total_carbon_emissions'].values[0]

pak_r = pak / (pak + bgd)
bgd_r = 1 - pak_r

select = (co2.name == 'EAST & WEST PAKISTAN')

ratio = {
    'pak': pak_r,
    'bgd': bgd_r
}

In [21]:
res = split_with_ratio(co2.ix[select], ratio)

In [22]:
co2 = co2.append(res, ignore_index=True)

In [23]:
# YUGOSLAVIA (MONTENEGRO & SERBIA):  split with 2006 ratio of mne and srb

mne = co2.ix[(co2.geo == 'mne') & (co2.year == 2006), 'total_carbon_emissions'].values[0]
srb = co2.ix[(co2.geo == 'srb') & (co2.year == 2006), 'total_carbon_emissions'].values[0]

mne_r = mne / (mne + srb)
srb_r = 1 - mne_r

ratio = {
    'mne': mne_r,
    'srb': srb_r
}

In [24]:
select = (co2.name == 'YUGOSLAVIA (MONTENEGRO & SERBIA)')
res = split_with_ratio(co2.ix[select], ratio)

In [25]:
co2 = co2.append(res, ignore_index=True)

In [26]:
def get_ratio(data, year, base):
    splits = dict()
    ratio = dict()
    for b in base:
        spl = data.ix[(data.geo == b) & (data.year == year), 'total_carbon_emissions'].values[0]
        splits[b] = spl
    
    sum_all = sum(splits.values())
    
    for b, value in splits.items():
        ratio[b] = value / sum_all
        
    return ratio

In [27]:
# CZECHOSLOVAKIA: split with 1992 ratio of cze and svk

ratio = get_ratio(co2, 1992, ['cze', 'svk'])

In [28]:
select = (co2.name == 'CZECHOSLOVAKIA')
res = split_with_ratio(co2.ix[select], ratio)

In [29]:
co2 = co2.append(res, ignore_index=True)

In [30]:
# USSR = ukr, rus, blr, arm, aze, est, geo, kaz, kgz, lva, ltu, mda, tjk, tkm, uzb on 1992

spls = ['ukr', 'rus', 'blr', 'arm', 'aze', 'est', 'geo', 'kaz', 'kgz', 'lva', 'ltu', 
        'mda', 'tjk', 'tkm', 'uzb']

ratio = get_ratio(co2, 1992, spls)

In [31]:
select = (co2.name == 'USSR')
res = split_with_ratio(co2.ix[select], ratio)

In [33]:
# only add rus

co2 = co2.append(res[res.geo == 'rus'], ignore_index=True)

Because we only set the geo for the merge cases, now we should group the data by geo/year and get aggregate values

In [34]:
co2_filtered = co2[~co2.geo.isnull()].copy()  # drop all rows don't have geo mappings.

In [35]:
co2_agg = co2_filtered.groupby(by=['geo', 'year'])['total_carbon_emissions'].sum()

In [36]:
co2_agg[:30]

geo  year
abw  1986     49.0
     1987    122.0
     1988    167.0
     1989    177.0
     1990    472.0
     1991    483.0
     1992    425.0
     1993    437.0
     1994    435.0
     1995    439.0
     1996    441.0
     1997    450.0
     1998    467.0
     1999    477.0
     2000    633.0
     2001    643.0
     2002    647.0
     2003    659.0
     2004    660.0
     2005    681.0
     2006    681.0
     2007    707.0
     2008    684.0
     2009    688.0
     2010    670.0
     2011    665.0
     2012    355.0
     2013    239.0
afg  1949      4.0
     1950     23.0
Name: total_carbon_emissions, dtype: float64

In [37]:
co2_agg = co2_agg.reset_index()

In [38]:
co2_agg.columns = ['country', 'year', 'total_carbon_emissions']

# calculation of indicators

full list see https://github.com/open-numbers/ddf--gapminder--co2_emission/issues/2

In [39]:
# Yearly CO2 emissions (1000 tonnes)

yearly_co2 = co2_agg.copy()

In [40]:
yearly_co2.head()

,country,year,total_carbon_emissions
0,abw,1986,49.0
1,abw,1987,122.0
2,abw,1988,167.0
3,abw,1989,177.0
4,abw,1990,472.0


In [41]:
yearly_co2.columns = ['country', 'year', 'yearly_co2_emissions_1000_tonnes']

In [42]:
yearly_co2 = yearly_co2.set_index(['country', 'year'])

In [43]:
# get CO2 emission

yearly_co2 = yearly_co2 * ((12+16*2)/12)

In [44]:
# set negative value to zero.

yearly_co2.ix[yearly_co2['yearly_co2_emissions_1000_tonnes'] < 0, 'yearly_co2_emissions_1000_tonnes'] = 0

In [45]:
# confirm no negative values
yearly_co2.ix[yearly_co2['yearly_co2_emissions_1000_tonnes'] < 0]

,,yearly_co2_emissions_1000_tonnes
country,year,


In [46]:
csv = yearly_co2.copy()

In [47]:
csv['yearly_co2_emissions_1000_tonnes'] = \
csv['yearly_co2_emissions_1000_tonnes'].map(format_float_digits)

In [48]:
csv.to_csv('../../ddf--datapoints--yearly_co2_emissions_1000_tonnes--by--country--year.csv')

In [49]:
# Cumulative CO2 emissions (tonnes)

cumulative_co2 = yearly_co2.sort_index(level=[0, 1]).groupby(level=0)

In [50]:
cumulative_co2 = cumulative_co2['yearly_co2_emissions_1000_tonnes'].apply(np.cumsum)

In [51]:
cumulative_co2 = cumulative_co2.reset_index()

In [52]:
cumulative_co2.columns = ['country', 'year', 'cumulative_co2_emissions_tonnes']

In [53]:
cumulative_co2.head()

,country,year,cumulative_co2_emissions_tonnes
0,abw,1986,179.666667
1,abw,1987,627.000000
2,abw,1988,1239.333333
3,abw,1989,1888.333333
4,abw,1990,3619.000000


In [54]:
# unit

cumulative_co2['cumulative_co2_emissions_tonnes'] = \
cumulative_co2['cumulative_co2_emissions_tonnes'] * 1000

In [55]:
csv = cumulative_co2.copy()

csv['cumulative_co2_emissions_tonnes'] = \
csv['cumulative_co2_emissions_tonnes'].map(int)

In [56]:
csv.to_csv('../../ddf--datapoints--cumulative_co2_emissions_tonnes--by--country--year.csv', index=False)

In [57]:
# CO2 per capita (tonnes per person)

pop.columns = ['country', 'year', 'value']
pop = pop.set_index(['country', 'year'])

In [58]:
a = yearly_co2.yearly_co2_emissions_1000_tonnes / pop['value']

In [59]:
per_person = a.dropna() * 1000

In [60]:
per_person = per_person.reset_index()
per_person.columns = ['country', 'year', 'co2_emissions_tonnes_per_person']

In [61]:
per_person.head()

,country,year,co2_emissions_tonnes_per_person
0,abw,1986,2.868059
1,abw,1987,7.234306
2,abw,1988,10.025596
3,abw,1989,10.633766
4,abw,1990,27.847504


In [62]:
csv = per_person.copy()

csv['co2_emissions_tonnes_per_person'] = csv['co2_emissions_tonnes_per_person'].map(format_float_digits)

In [63]:
csv.to_csv('../../ddf--datapoints--co2_emissions_tonnes_per_person--by--country--year.csv', index=False)

# concepts

In [49]:
concepts = [
    'name',
    'indicator_url',
    'unit',
    'country',
    'year',
    'yearly_co2_emissions_1000_tonnes',
    'cumulative_co2_emissions_tonnes',
    'co2_emissions_tonnes_per_person'
]

In [57]:
cdf = pd.DataFrame(concepts, columns=['concept'])

In [58]:
cdf['name'] = [
    'Name',
    'Indicator URL',
    'Unit',
    'Country',
    'Year',
    'Yearly CO2 emissions',
    'Cumulative CO2 emissions',
    'CO2 per capita'
]

In [59]:
cdf = cdf.set_index('concept')

In [60]:
cdf['concept_type'] = 'measure'
cdf.ix[['name', 'indicator_url', 'unit'], 'concept_type'] = 'string'
cdf.ix['country', 'concept_type'] = 'entity_domain'
cdf.ix['year', 'concept_type'] = 'time'

In [61]:
cdf.ix[cdf.concept_type == 'measure', 'indicator_url'] = 'https://github.com/open-numbers/ddf--gapminder--co2_emission'

In [62]:
cdf.ix['yearly_co2_emissions_1000_tonnes', 'unit'] = '1000 metric tons'
cdf.ix['cumulative_co2_emissions_tonnes', 'unit'] = 'metric tons'
cdf.ix['co2_emissions_tonnes_per_person', 'unit'] = 'metric tons per person'

In [63]:
cdf

,name,concept_type,indicator_url,unit
concept,,,,
name,Name,string,NaN,NaN
indicator_url,Indicator URL,string,NaN,NaN
unit,Unit,string,NaN,NaN
country,Country,entity_domain,NaN,NaN
year,Year,time,NaN,NaN
yearly_co2_emissions_1000_tonnes,Yearly CO2 emissions,measure,https://github.com/open-numbers/ddf--gapminder...,1000 mertic tons
cumulative_co2_emissions_tonnes,Cumulative CO2 emissions,measure,https://github.com/open-numbers/ddf--gapminder...,mertic tons
co2_emissions_tonnes_per_person,CO2 per capita,measure,https://github.com/open-numbers/ddf--gapminder...,metric tons per person


In [64]:
cdf.to_csv('../../ddf--concepts.csv')

# entity

In [65]:
# just copy the GM geo domain

geo[['country', 'name']].to_csv('../../ddf--entities--country.csv', index=False)

In [158]:
!validate-ddf ../../

[
{}]

